In [23]:
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_groq import ChatGroq



In [22]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")
genai.configure(api_key=api_key)


In [9]:
file_path="Corpus.pdf"
loader=PyPDFLoader(file_path)

docs=loader.load()

print(len(docs))

10


In [24]:
llm = ChatGroq(model="llama3-8b-8192")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)


In [25]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits=text_splitter.split_documents(docs)

texts = [docs.page_content for docs in splits]
vector_store = FAISS.from_texts(texts, embedding=embeddings)
retriever=vector_store.as_retriever()

In [30]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If the question is out of the context, reply this statement only "
    "For this information, contact us directly."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [31]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [32]:

results = rag_chain.invoke({"input": "Who is the pm of India?"})
results

{'input': 'Who is the pm of India?',
 'context': [Document(page_content='south\nof\nSt.\nHelena\nin\nthe\ncenter\nof\nthe\nNapa\nValley.\nYou\ncan\nreach\nYountville\nby\ncar\nfrom\nanywhere\nin\nthe\nNorthern\nCalifornia\nBay\nArea\nin\nabout\nan\nhour,\nwhile\ntravelers\nfrom\nthe\nSacramento\nregion\nwill\ntake\nabout\n1.5\nhours.\nThe\narea\nis\nserved\nby\nmultiple\nairports\nincluding\nSFO,\nOakland,\nSacramento\nand\nSanta\nRosa.\nJessup\nCellars\nwelcomes\nyour\nwell-behaved\ndogs\ninside\nor\noutside\nand\nwe\nhave\ngluten\nfree\ndog\ntreats\navailable\nas\nwell\nas\nwater\ndishes.\nJessup\nCellars\nwines\nare\ncarefully\ncrafted\nwith\nthe\nhelp\nof\nour\nrenowned\nconsulting\nwinemaker\nRob\nLloyd\nwho\nfamously\ncrafted\nChardonnay\nfor\nRombauer,\nLa\nCrema\nand\nCakebread.\nNot\nonly\nhas\nRob\ncreated\none\nof\nthe\nbest\nChardonnays\nin\nthe\nNapa\nValley\nwith\nour\n2022'),
  Document(page_content="winemakers\nwith\nJessup\noffering\nour\n2019\nCabernet\nSauvignon\nfor